This code is designed to measure the synchronisation of the signal

In [1]:
!pip install sounddevice



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Maxle\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import sounddevice as sd


In [3]:

import numpy as np
import threading
import time

# --- Configuration ---
# Adjust these parameters based on your microphone and environment

SAMPLERATE = 44100  # Samples per second (common values: 44100, 48000)
BLOCKSIZE = 1024    # Number of frames passed to the callback function
THRESHOLD = 0.05    # Amplitude threshold for clap detection (adjust as needed)
CLAP_HOLD_FRAMES = 5 # Number of consecutive blocks exceeding threshold to confirm clap
DEVICE = None       # None for default input device, or specify device index/name

# --- State Variables ---
clap_detected = False
frames_since_threshold = 0
stream = None
stop_event = threading.Event()

# --- Callback Function ---
def audio_callback(indata, frames, time_info, status):
    """This is called (from a separate thread) for each audio block."""
    global clap_detected, frames_since_threshold

    if status:
        print(status)

    # Calculate the amplitude (RMS or peak)
    amplitude = np.linalg.norm(indata) / np.sqrt(frames) # RMS amplitude

    if amplitude > THRESHOLD:
        frames_since_threshold += 1
        if frames_since_threshold >= CLAP_HOLD_FRAMES and not clap_detected:
            print("Clap detected!")
            clap_detected = True
    else:
        frames_since_threshold = 0
        # Optional: Reset clap_detected after a certain period of low amplitude
        if clap_detected and amplitude < THRESHOLD * 0.5: # Reset if amplitude drops significantly
             clap_detected = False


# --- Main Function to Start and Stop the Stream ---
def start_clap_detector(samplerate=SAMPLERATE, blocksize=BLOCKSIZE, device=DEVICE, threshold=THRESHOLD, clap_hold_frames=CLAP_HOLD_FRAMES):
    """Starts the audio stream to listen for claps."""
    global stream, clap_detected, frames_since_threshold, stop_event

    clap_detected = False
    frames_since_threshold = 0
    stop_event.clear()

    try:
        stream = sd.InputStream(
            samplerate=samplerate,
            blocksize=blocksize,
            device=device,
            channels=1,  # Use 1 for mono audio
            dtype='float32', # Data type
            callback=audio_callback
        )
        print("Starting audio stream. Listening for claps...")
        stream.start()

    except Exception as e:
        print(f"Error starting stream: {e}")
        stream = None

def stop_clap_detector():
    """Stops the audio stream."""
    global stream, stop_event
    if stream is not None and stream.running:
        print("Stopping audio stream.")
        stop_event.set()
        stream.stop()
        stream.close()
        stream = None
        print("Stream stopped.")
    else:
        print("Stream is not running.")

In [7]:
# To start listening:
start_clap_detector()

# Keep the notebook alive while listening.
# You can run this in a separate cell after starting the detector.
# This loop will block the cell execution but allows the callback to run.
# You can interrupt this cell's execution to stop the listening loop,
# but it's better to use the stop_clap_detector() function.

# while not stop_event.is_set() and stream is not None and stream.running:
#     time.sleep(0.1)

print("Clap detection code loaded. Use start_clap_detector() to begin listening.")

Starting audio stream. Listening for claps...
Clap detection code loaded. Use start_clap_detector() to begin listening.


In [ ]:
# To stop listening:
stop_clap_detector()

AttributeError: 'InputStream' object has no attribute 'running'

: 